In [1]:
import pennylane as qml
import pennylane.numpy as np

In [2]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import matplotlib as plt
import itertools as it
from itertools import product
from matplotlib import pyplot as plt
import math

import matplotlib.pyplot as plt
import networkx as nx

In [4]:
def next_to(p,n,T):
    if T == 'Default':
        py = int(p % n) # the y coordinate (y in {0,...,n-1})
        px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
        # print((px,py))
        adjacent_set = []
        if px + 1 < n and px + 1 >= 0:
            adjacent_set.append((px + 1, py))
        if px - 1 < n and px - 1 >= 0:
            adjacent_set.append((px - 1, py))
        if py + 1 < n and py + 1 >= 0:
            adjacent_set.append((px, py + 1))
        if py - 1 < n and py - 1 >= 0:
            adjacent_set.append((px, py - 1)) 
        tensors = []
        neighbours = []
        for x,y in adjacent_set:
            tensors.append(f'G_{x*n + y}')
            neighbours.append(x*n + y)
        return neighbours
    
    elif T == 'Periodic':
        py = int(p % n) # the y coordinate (y in {0,...,n-1})
        px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
        # print((px,py))
        adjacent_set = []
        adjacent_set.append(((px + 1) % n, py))
        adjacent_set.append(((px - 1) % n, py)) 
        adjacent_set.append((px, (py + 1) % n))
        adjacent_set.append((px, (py - 1) % n)) 
        tensors = []
        neighbours = []
        # print(f'G_{p}')
        for x,y in adjacent_set:
            # print((px,py),f'G_{x*n + y}')
            tensors.append(f'G_{x*n + y}')
            neighbours.append(x*n + y)

        return neighbours


    elif T == 'X Shift':
        shift = 1
        py = int(p % n) # the y coordinate (y in {0,...,n-1})
        px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
        # print((px,py))
        adjacent_set = []
        

        #Bulk
        if px not in [0,n-1]:
            adjacent_set.append(((px + 1) % n, py))
            adjacent_set.append(((px - 1) % n, py)) 
            adjacent_set.append((px, (py + 1) % n))
            adjacent_set.append((px, (py - 1) % n)) 
            # print('Bulk point: ',adjacent_set)
        #x lower boundary
        elif px == 0:
            adjacent_set.append(((px + 1) % n, py))
            adjacent_set.append(((px - 1) % n, (py - shift) % n)) 
            adjacent_set.append((px, (py + 1) % n))
            adjacent_set.append((px, (py - 1) % n))
            # print('lower x point: ', adjacent_set)
        #x upper boundary
        elif px == n-1:
            adjacent_set.append(((px + 1) % n, (py + shift) % n))
            adjacent_set.append(((px - 1) % n, py % n)) 
            adjacent_set.append((px, (py + 1) % n))
            adjacent_set.append((px, (py - 1) % n))
            # print('upper x point: ', adjacent_set)
        tensors = []
        neighbours = []
        # print(f'G_{p}')
        for x,y in adjacent_set:
            # print((px,py),f'G_{x*n + y}')
            tensors.append(f'G_{x*n + y}')
            neighbours.append(x*n + y)

        return neighbours

# Convert to coordiante form
def coords(p,n):
    p = int(p)
    py = int(p % n) # the y coordinate (y in {0,...,n-1})
    px = int((p - py)/n) # the x coordinate (x in {0,...,n-1})
    return (px,py)

# Get adjacent Edges for each point
def edge_set(p,n):
    neighbours = next_to(p,n, T = BC)
    edge_set = []
    edge_tensors = []
    for v in neighbours:
        if v < p:
            edge_set.append((v,p))
            edge_tensors.append(f'G_{v}_{p}')
        else:
            edge_set.append((p,v))
            edge_tensors.append(f'G_{p}_{v}')
    return edge_set

# Get all bonds globally
def get_all_bonds_global(n):
    bonds = []
    for j in range(n**2):
        for p in next_to(j,n, T = BC):
            bonds.append(bond_map(f'G_{j}_{p}'))
        bonds = list(set(bonds))
    return bonds

# Get all local bonds
def get_all_bonds(j,n):
    bonds = []
    for p in next_to(j,n, T = BC):
        bonds.append(bond_map(f'G_{j}_{p}'))
    bonds = list(set(bonds))
    return bonds

# Bond map G_j_i -> G_i_j where i < j
def bond_map(bond):
    # print(bond)
    G,i,j = bond.split('_')
    if int(i) < int(j):
        return f'G_{int(i)}_{int(j)}'
    else:
        return f'G_{int(j)}_{int(i)}'

# Decompose bonds
def decompose_bond(bond):
    
    A = bond.split('_')
    i = A[1]
    j = A[2]
    decomp = [f'G_{i}',f'G_{j}']
    return decomp

# Corner map G_i_j_k -> G_k_j_i wherer k < i
def corner_map(bond):
    G,i,j,k = bond.split('_')
    if int(i) < int(k):
        return f'G_{int(i)}_{int(j)}_{int(k)}'
    else:
        return f'G_{int(k)}_{int(j)}_{int(i)}'

# Get corner tensors around a point, this will account for all the corners in a plus shape
def get_corners(j,n):
    neighbours = next_to(j,n, T = BC)
    h_list = [p for p in neighbours if (p % n) == (j % n)]
    v_list = [p for p in neighbours if (p % n) != (j % n)]
    corners = [corner_map(f'G_{v}_{j}_{h}') for (v,h) in it.product(h_list,v_list)]
    return corners

# Get all rank 3 tensors surroundng a point.
def get_all_3_tensors(j,n):
    neighbours = next_to(j,n, T = BC)
    _3_tensors = []
    pairs = [(i,k) for i,k in it.combinations(neighbours,2)]
    for (i,k) in pairs:
        _3_tensors.append(f'G_{i}_{j}_{k}')
    return _3_tensors

# Split corners up into rank two tensors
def decompose_corners(corner):
    G,i,j,k = corner.split('_')
    decomp = [bond_map(f'G_{i}_{j}'),bond_map(f'G_{j}_{k}')]
    return decomp

# Rank 4 tensor map
def _4_tensor_map(tensor):
    parts = tensor.split('_')[1:]
    if int(parts[0]) > int(parts[-1]):
        parts.reverse()
    return(f'G_{parts[0]}_{parts[1]}_{parts[2]}_{parts[3]}')

# Get all rank 4 tensors
def get_4_tensors(j,n):
    tensors = []
    neighbours = next_to(j,n, T = BC)
    d2_neighbours =[next_to(i,n) for i in neighbours]
    for (a,b) in [(a,b) for a,b in it.combinations(neighbours,2)]:
        for k in next_to(b,n,T = BC):
            tensors.append(_4_tensor_map(f'G_{a}_{j}_{b}_{k}'))
    return list(set(tensors))

# Decompose all rank 4 tensors
def decompose_4_tensors(j,n):
    G,i,j,k,l = corner.split('_')
    decomp = [corner_map(f'G_{i}_{j}_{k}'),corner_map(f'G_{j}_{k}_{l}')]
    return decomp

def plot_solution(active_coords,active_edges,n):
    n_ = n  # Size of the square lattice

    # Create a 2D grid graph
    G = nx.Graph()
    for i in active_coords:
        G.add_node(i)

    for u, v in [(i, j) for i,j in active_edges]:
        G.add_edge(u, v)

    # Draw the nodes and edges of the graph
    pos = {(i, j): (i, j) for i in range(n_) for j in range(n_)}
    nx.draw(G, pos=pos, with_labels=True, node_size=10, edge_color='gray')

    # Show the plot
    plt.axis('equal')
    plt.show()

def checking_function(N,L,sites,bonds,rank_three_tensors):
    # Check there are the correct number of sites
    if N == len(sites):
        print('Correct number of sites:', N)
    else:
        print('Worng number of sites: ', len(sites))
    # Check there are the correct number of bonds
    if L == len(bonds):
        print('Correct number of bonds:', L)
    else:
        print('Worng number of bonds: ', len(bonds))

    # Check that there are 2 sites for every bond
    for bond in bonds:
        sites_in_bond = []
        for site in sites:
            if set(bond).issuperset(set(site)):
                sites_in_bond.append(tuple(site))
                # print(site,bond,len(set(tuple(sites_in_bond))))
        if len(set(tuple(sites_in_bond))) != 2:
            print('Bond : ' , bond, 'is not consistent.','The sites in this bond are: ',sites_in_bond)

    # Check there is a bond on every site
    for site in sites:
        bonds_for_site = []
        for bond in bonds:
            if set(site).issubset(set(bond)):
                bonds_for_site.append(tuple(bond))
        if len(set(bonds_for_site)) != 1 and len(set(tuple(bonds_for_site))) != 2:
            print('Site :',site, f'is part of {len(set(tuple(bonds_for_site)))} bonds')


    # Check there are two bonds in every tensor
    for t3 in rank_three_tensors:
        for bond in bonds:
            bond_in_tensor = []
            if set(t3).issuperset(set(bond)):
                bond_in_tensor.append(tuple(bond))
            if len(set(bond_in_tensor)) > 2:
                print('Tensor :', t3, 'is not consistent with bonds : ', bond)

    for bond in bonds: 
        tensor_containing_bond = []
        for t3 in rank_three_tensors:
            if set(bond).issubset(t3):
                tensor_containing_bond.append(tuple(t3))
        if len(set(tensor_containing_bond)) != 1 and len(set(tensor_containing_bond)) != 2:
            print('Bond :', bond, f'is part of {tensor_containing_bond} bonds')

    
    # Check that there are bonds for all rank 3 tensors
    for t in rank_three_tensors:
        t = corner_map(f'G_{t[0]}_{t[1]}_{t[2]}')
        bond1, bond2 = decompose_corners(t)
        bond1 = bond1.split('_')[1:]
        bond2 = bond2.split('_')[1:]
        if (bond1 not in bonds) or (bond2 not in bonds):
            print(f'bond {bond1} or {bond2} are not contained within bonds but are in {t}' )


In [3]:
def create_Hamiltonian(n,N,L,Am,Ab,Asa,A2,A3,A4,L4,L6,BC):
    '''
    n is the size of the grid
    N is the number of active sites
    L is the number of bonds
    Am is the monomer strength
    Ab is the bond strength
    A2 is the rank 1-2 tensor consistency bias
    Asa is the self avoiding walk bias 
    A3 is the rank 2-3 tensor consistency bias
    '''

    

SyntaxError: expected ':' (2014863917.py, line 1)

In [6]:
d = {'a' : 1}

In [9]:
d['a'] += 1

In [10]:
d

{'a': 2}